In [2]:
import keras
import numpy as np
from keras import layers
import pandas as pd
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization,Flatten,Conv2D
from keras.layers import AveragePooling2D,MaxPooling2D,Dropout, GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as k


Using TensorFlow backend.
C:\Users\Rittik Grignard\Anaconda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Rittik Grignard\Anaconda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Rittik Grignard\Anaconda\envs\deeplearning\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])

In [3]:

data=pd.read_csv(r'C:\Users\Rittik Grignard\Downloads\data1\data.csv')
dataset=np.array(data)
np.random.shuffle(dataset)
x=dataset
y=dataset
x=x[:,0:1024]
y=y[:,1024]
x_train=x[0:70000,:]
x_train=x_train/255.
x_test=x[70000:72001,:]
x_test=x_test/255.
x_train=np.asarray(x_train)
x_test=np.asarray(x_test)
y=y.reshape(y.shape[0],1)
y_train=y[0:70000,:]
y_train=y_train.T
y_test=y[70000:72001,:]
y_test=y_test.T

print(x_train.shape[0])

print(x_test.shape[0])
image_x=32
image_y=32

train_y=np_utils.to_categorical(y_train)
test_y=np_utils.to_categorical(y_test)
train_y=np.asarray(train_y)
test_y=np.asarray(test_y)

train_y=train_y.reshape(train_y.shape[1],train_y.shape[2])
test_y=test_y.reshape(test_y.shape[1],test_y.shape[2])
x_train=x_train.reshape(x_train.shape[0],image_x,image_y,1)
x_test=x_test.reshape(x_test.shape[0],image_x,image_y,1)
print("x train shape:"+str(x_train.shape))
print("x test shape:"+str(x_test.shape))

70000
2000
x train shape:(70000, 32, 32, 1)
x test shape:(2000, 32, 32, 1)


In [38]:
#Building the model
def keras_model(image_x,image_y):
    num_of_classes=37
    model=Sequential()
    model.add(Conv2D(filters=32,kernel_size=(5,5),input_shape=(image_x,image_y,1),activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same'))
    model.add(Conv2D(64,(5,5),activation='relu'))
    model.add(MaxPooling2D(pool_size=(5,5),strides=(5,5),padding='same'))
    model.add(Flatten())
    model.add(Dense(num_of_classes,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    filepath="devanagari.h5"
    checkpoint1=ModelCheckpoint(filepath,monitor='val_acc',verbose=1, save_best_only=True,mode='max')
    callbacks_list=[checkpoint1]
    return model,callbacks_list

In [39]:
model,callbacks_list=keras_model(image_x,image_y)
model.fit(x_train,train_y,validation_data=(x_test,test_y),epochs=2,batch_size=64,callbacks=callbacks_list)
scores=model.evaluate(x_test,test_y,verbose=0)
print("CNN:Error%.2f%%"%(100-scores[1]*100))
print_summary(model)
model.save('devanagari.h5')


Train on 70000 samples, validate on 2000 samples
Epoch 1/2
70000/70000 [==============================] - 359s 5ms/step - loss: 0.7774 - accuracy: 0.7791 - val_loss: 0.2910 - val_accuracy: 0.9175
Epoch 2/2
   64/70000 [..............................] - ETA: 3:13 - loss: 0.2740 - accuracy: 0.9375

C:\Users\Rittik Grignard\Anaconda\envs\deeplearning\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


70000/70000 [==============================] - 131s 2ms/step - loss: 0.2251 - accuracy: 0.9317 - val_loss: 0.1899 - val_accuracy: 0.9460
CNN:Error5.40%
Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 10, 10, 64)        51264     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None